In [ ]:
import re
from warnings import warn

In [ ]:
TEST = False

In [ ]:
if TEST:
    filename = "data/input_7_test"
else:
    filename = "data/input_7"

In [ ]:
with open(filename) as file:
    input_str = file.read()

In [ ]:
terminal = input_str.strip("\n").split("\n")

In [ ]:
terminal[-10:]

In [ ]:
class Tree:
    
    def __init__(self, name, kind, parent, size=None):
        self.name = name
        self.kind = kind
        self.ref = self.kind +'-'+self.name
        self.parent = parent
        self.children = {}
        self.size = size
    
    def add_child(self, tree):
        '''Add a child tree'''
        if self.kind == 'dir':
            if tree.ref in self.children.keys():
                warn("Attempt to add an existing child {0} to {1} - stopped".format(tree.name, self.name))
            else:
                self.children[tree.ref] = tree
        else:
            raise TypeError('Trying to add files to a non-directory')
    
    def get_child(self, child_ref):
        '''Return child by name'''
        try:
            child = self.children[child_ref]
        except KeyError:
            raise KeyError('Tree {0} has no child {1}'.format(self.name, child_ref)) from None
        return child
    
    def get_all_dirs(self):
        if self.kind=='file':
            dirs = []
        else:
            dirs = [(self.name, self.size)]
            for child in self.children.values():
                dirs += child.get_all_dirs()
        return dirs
    
    def render_tree(self, n=0):
        if self.kind=='file':
            if self.size is not None:
                rep = "{2}- {0} ({1}, size={3})".format(self.name, self.kind, n*'  ', self.size)
            else:
                rep = "{2}- {0} ({1})".format(self.name, self.kind, n*'  ')
            return rep
        else:
            if self.size is not None:
                rep = "{2}- {0} ({1}, size={3})".format(self.name, self.kind, n*'  ', self.size)
            else:
                rep =  "{2}- {0} ({1})".format(self.name, self.kind, n*'  ')
            for child in self.children.values():
                rep = rep + '\n{0}'.format((n+1)*'  ' + child.render_tree(n=n+1))
            return rep
    
    def get_size(self):
        if self.kind=='dir':
            self.size = sum([child.get_size() for child in self.children.values()])
        return self.size
    
    def __repr__(self):
        return self.render_tree()
    
    def __str__(self):
        return self.render_tree()
    

In [ ]:
def build_tree(terminal, get_size=True, verbose=False):
    file_system = Tree("/", "dir", None)
    cwd = file_system

    for line in terminal[1:]:
        if verbose:
            print(line)
            print("  cwd = {0}".format(cwd.name))
        if line[0]=="$": # Command
            if line[2:4]=='cd':
                if line[5:7]=='..': # go up a level
                    if verbose: print('  going up a level')
                    cwd = cwd.parent
                else: # go into child directory
                    if verbose: print('  going down a level into {0}'.format(cwd.get_child('dir-' + line[5:]).name))
                    cwd = cwd.get_child('dir-' + line[5:])
            else: # ls command
                if verbose: 
                    print("  listing files in {0}".format(cwd.name))
                else:
                    continue 
        elif line[:3]=='dir': # ls output: directory
            name = line.split(" ")[-1]
            new_tree = Tree(name, 'dir', cwd)
            cwd.add_child(new_tree)
            if verbose: print("  Addind new directory {0} to current directory {1}"
                              .format(cwd.get_child('dir-' + name).name, cwd.name))
        else: # ls output: file
            size, name  = line.split(" ")
            new_tree = Tree(name, 'file', cwd, size=int(size))
            cwd.add_child(new_tree)
            if verbose: print("  Adding new file {0} to current directory {1}"
                              .format(cwd.get_child('file-' + name).name, cwd.name))
    if get_size:
        _ = file_system.get_size()
        
    return file_system

In [ ]:
file_system = build_tree(terminal, get_size=True, verbose=False)

In [ ]:
# file_system

In [ ]:
file_system.get_size()

In [ ]:
all_dirs = file_system.get_all_dirs()

Part 1  - Find all small directories

In [ ]:
small_dirs = [directory for directory in all_dirs if directory[1]<=100000]

In [ ]:
small_dirs_sum = sum([directory[1] for directory in small_dirs])

In [ ]:
TEST_ANSWER = 95437

In [ ]:
if TEST:
    assert small_dirs_sum == TEST_ANSWER
else: 
    print("total memory in small directories is {0}".format(small_dirs_sum))

Part 2 - which directory to delete?

In [ ]:
total_space = 70000000
update_size = 30000000

In [ ]:
available_space = total_space - file_system.get_size()
space_needed  = update_size - available_space
print("We have {0} available space, so we need to free up {1} more space"
      .format(available_space, space_needed))

In [ ]:
candidate_dirs = [directory for directory in all_dirs if directory[1]>=space_needed]
dir_to_delete = min(candidate_dirs, key = lambda t: t[1])

In [ ]:
TEST_ANSWER = 24933642

In [ ]:
if TEST:
    assert dir_to_delete[1] == TEST_ANSWER
else: 
    print("directory to delete is {0}".format(dir_to_delete))